## ホームベースを認識させるための処理

##### 学習データセットを作成するために、映像データからホームベースが映っているフレーム（最初の1枚）を自動的に切り出す。

In [14]:
import cv2
import os
import glob

# ==============================================================================
# 設定エリア
# ==============================================================================
# 動画データが格納されているディレクトリ
VIDEO_DIR = 'video'
# 切り出した画像の保存先ディレクトリ
SAVE_DIR = 'capture'
# 保存する画像の最大枚数（教師データの作成コストと学習効率のバランスを考慮して設定）
MAX_IMAGES = 30  

# 保存先ディレクトリが存在しない場合は作成する
os.makedirs(SAVE_DIR, exist_ok=True)


# ==============================================================================
# 1. 動画ファイルリストの取得とソート
#    OSによってファイルの取得順序が異なるため、ファイル名の数値順に正しくソートする
# ==============================================================================
video_files = glob.glob(os.path.join(VIDEO_DIR, "*.mp4"))

try:
    # ファイル名（例: "1.mp4"）から拡張子を除き、数値としてソートする
    video_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
    print("ファイル名を数値順（1, 2, 3...）に並び替えました。")
except ValueError:
    # 数値以外のファイル名が含まれる場合は、通常の文字列順でソート
    print("注意: ファイル名に数値以外が含まれているため、名前順（文字順）で処理します。")
    video_files.sort()

print(f"全動画数: {len(video_files)}本")


# ==============================================================================
# 2. データのサンプリング（間引き抽出）
#    データセットには解像度の異なる動画（720p, 1080p）が混在している可能性があるため、
#    先頭から順に取得するのではなく、全データから等間隔に抽出することで
#    学習データの多様性（バリエーション）を確保する。
# ==============================================================================

# ステップ数を計算 (例: 150本 ÷ 30枚 = 5本おきに抽出)
step = len(video_files) // MAX_IMAGES
if step == 0: step = 1

# スライス機能を使って等間隔にリストから抽出
target_files = video_files[::step]

# 念のため、指定枚数ちょうどになるようにリストを切り詰める
target_files = target_files[:MAX_IMAGES]

print(f"抽出対象: {len(target_files)}本 (全データから等間隔にサンプリングしました)")
# 確認用に最初の5件を表示
print(f"抽出ファイル例: {target_files[:5]} ...")


# ==============================================================================
# 3. 画像切り出し処理の実行
#    各動画の「最初の1フレーム」を静止画として保存する。
#    ホームベースは静止物体であるため、動画全体ではなく1枚の画像で十分な学習が可能
# ==============================================================================
count = 0

print("--- 画像切り出し開始 ---")

for video_path in target_files:
    # 動画ファイルを読み込む
    cap = cv2.VideoCapture(video_path)
    
    # 最初の1フレームだけを読み込む
    ret, frame = cap.read() 
    
    if ret:
        # 保存ファイル名を生成 (例: capture_1.jpg)
        # "capture_" を付けることで、ボール画像（数字のみ）と区別しやすくする
        base_name = os.path.basename(video_path).split('.')[0]
        save_path = os.path.join(SAVE_DIR, f"capture_{base_name}.jpg")
        
        # 画像ファイルとして書き出し
        cv2.imwrite(save_path, frame)
        print(f"保存完了: {save_path}")
        count += 1
    else:
        print(f"警告: 動画の読み込みに失敗しました ({video_path})")
    
    # メモリ解放
    cap.release()

print("-" * 30)
print(f"処理完了: 合計 {count} 枚の画像を切り出しました。")
print(f"保存先ディレクトリ: {os.path.abspath(SAVE_DIR)}")

ファイル名を数値順（1, 2, 3...）に並び替えました。
全動画数: 150本
抽出対象: 30本 (全データから等間隔にサンプリングしました)
抽出ファイル例: ['video/1.mp4', 'video/6.mp4', 'video/11.mp4', 'video/16.mp4', 'video/21.mp4'] ...
--- 画像切り出し開始 ---
保存完了: capture/capture_1.jpg
保存完了: capture/capture_6.jpg
保存完了: capture/capture_11.jpg
保存完了: capture/capture_16.jpg
保存完了: capture/capture_21.jpg
保存完了: capture/capture_26.jpg
保存完了: capture/capture_31.jpg
保存完了: capture/capture_36.jpg
保存完了: capture/capture_41.jpg
保存完了: capture/capture_46.jpg
保存完了: capture/capture_51.jpg
保存完了: capture/capture_56.jpg
保存完了: capture/capture_61.jpg
保存完了: capture/capture_66.jpg
保存完了: capture/capture_71.jpg
保存完了: capture/capture_76.jpg
保存完了: capture/capture_81.jpg
保存完了: capture/capture_86.jpg
保存完了: capture/capture_91.jpg
保存完了: capture/capture_96.jpg
保存完了: capture/capture_101.jpg
保存完了: capture/capture_106.jpg
保存完了: capture/capture_111.jpg
保存完了: capture/capture_116.jpg
保存完了: capture/capture_121.jpg
保存完了: capture/capture_126.jpg
保存完了: capture/capture_131.jpg
保存完了: capture/capture_136.jp